In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
master_folder = "/home/charly/Desktop/Wunderman/mazda/MailChimp/"
lists = "lists/"
granular_activity = "granular_activity/"
geo_data = "geo_data/"
campaigns_content = "campaigns_content/"
aggregate_activity = "aggregate_activity/"
campaigns = "campaigns.csv"

In [4]:
# Functions to count the number of files per folder
import os, os.path
def filecount(dir_name):
    return len([f for f in os.listdir(dir_name) if os.path.isfile(dir_name +f)])

In [5]:
# Load the mazda datasets
df_cust_data = pd.read_excel('mazda_dataset.xlsx','Customer data')
df_prospects_data = pd.read_excel('mazda_dataset.xlsx','Prospects data')

In [6]:
# Clean and format well the email columns where the joins will happen
df_cust_data['Email Address'] = df_cust_data['Email Address'].fillna("no email").astype(str)
df_cust_data['Email Address'] = df_cust_data['Email Address'].apply(lambda x: str(x.lower().strip()))
df_cust_data['Email Address'] = df_cust_data['Email Address'].astype(str)

df_prospects_data['Email Address'] = df_prospects_data['Email Address'].fillna("no email").astype(str)
df_prospects_data['Email Address'] = df_prospects_data['Email Address'].apply(lambda x: str(x.lower().strip()))
df_prospects_data['Email Address'] = df_prospects_data['Email Address'].astype(str)

## Campaign Master File

In [7]:
df_campaigns = pd.read_csv(master_folder + campaigns)

In [8]:
# Transform percentage into float
df_campaigns['Open Rate'] = df_campaigns['Open Rate'].apply(lambda x: x[:-1]).astype('float')
df_campaigns['Click Rate'] = df_campaigns['Click Rate'].apply(lambda x: x[:-1]).astype('float')

In [9]:
len(df_campaigns['Title']) == len(set(df_campaigns['Title']))

True

In [10]:
df_campaigns.describe()

,Total Recipients,Successful Deliveries,Soft Bounces,Hard Bounces,Total Bounces,Times Forwarded,Forwarded Opens,Unique Opens,Open Rate,Total Opens,Unique Clicks,Click Rate,Total Clicks,Unsubscribes,Abuse Complaints,Times Liked on Facebook,Folder Id
count,182.000000,182.000000,182.000000,182.000000,182.000000,182.0,182.0,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.0,182.000000
mean,1887.247253,1767.478022,23.824176,95.945055,119.769231,0.0,0.0,884.456044,53.958462,1852.395604,238.538462,16.065055,317.967033,10.065934,0.263736,0.0,646.549451
std,2917.957868,2741.280571,42.223843,153.697907,194.321405,0.0,0.0,1306.326728,21.013667,2612.225997,373.117146,18.204829,523.763315,16.428517,0.791323,0.0,1305.584709
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,71.250000,69.750000,0.000000,1.000000,1.000000,0.0,0.0,41.000000,44.685000,96.000000,6.000000,6.290000,5.000000,0.000000,0.000000,0.0,0.000000
50%,603.500000,573.500000,7.000000,25.000000,32.000000,0.0,0.0,341.500000,52.975000,688.500000,75.000000,12.990000,94.000000,3.000000,0.000000,0.0,0.000000
75%,2679.250000,2487.000000,26.000000,117.000000,147.250000,0.0,0.0,1203.250000,63.287500,2570.500000,285.750000,20.105000,340.000000,11.750000,0.000000,0.0,13.000000
max,18030.000000,16896.000000,239.000000,911.000000,1134.000000,0.0,0.0,7670.000000,100.000000,13894.000000,2127.000000,100.000000,2868.000000,92.000000,6.000000,0.0,3777.000000


In [11]:
df_campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 23 columns):
Title                      182 non-null object
Subject                    171 non-null object
List                       182 non-null object
Send Date                  182 non-null object
Send Weekday               182 non-null object
Total Recipients           182 non-null int64
Successful Deliveries      182 non-null int64
Soft Bounces               182 non-null int64
Hard Bounces               182 non-null int64
Total Bounces              182 non-null int64
Times Forwarded            182 non-null int64
Forwarded Opens            182 non-null int64
Unique Opens               182 non-null int64
Open Rate                  182 non-null float64
Total Opens                182 non-null int64
Unique Clicks              182 non-null int64
Click Rate                 182 non-null float64
Total Clicks               182 non-null int64
Unsubscribes               182 non-null int64
Abuse Compla

**Columns explanation:**
- Title: Title of the campaign
- Subject: Subject of the campaign
- List: Family of the campaign
- Total Recipients - Successful Deliveries = Total Bounces
- Soft Bounces: typically indicate a temporary delivery issue to an address 
- Hard Bounces: indicates a permanent reason an email cannot be delivere
- Times Forwarded: Number of times the campaign was forwarded
- Unique Opens: Number of times the campaign was opened (if a user opens two times the campaign, counts only as one)
- Open Rate = Unique_Opens / Successfull_Deliveries
- Total Opens: Total number of times the campaign was opened by recipients. This count includes multiple opens from individual recipients.
- Unique Clicks: Number of clicks (if a user clicks two times, counts only as one)
- Click Rate: Unique_Clicks / Successfull_Deliveries
- Total Clicks: Total number of times tracked links were clicked by recipients. This count includes multiple clicks from individual recipients
- Unsubscribes: Number of subscribers that opted out of your list using the unsubscribe link in this campaign.
- Abuse Complaints: Number of recipients that reported your email as spam. Subscribers that report a campaign as spam are automatically unsubscribed from your list.
- Folder Id: ?
- Unique Id: should be the foreign key to other files. Not the case for the moment.

In [12]:
df_campaigns.head()

,Title,Subject,List,Send Date,Send Weekday,Total Recipients,Successful Deliveries,Soft Bounces,Hard Bounces,Total Bounces,...,Open Rate,Total Opens,Unique Clicks,Click Rate,Total Clicks,Unsubscribes,Abuse Complaints,Times Liked on Facebook,Folder Id,Unique Id
0,Mazda - MAZDHOC7W495 - Newsletter Januari - FR...,E-news janvier 2018 : Conditions salon + 3 nou...,Mazda Newsletter Januari FR,"Jan 12, 2018 06:51 pm",Friday,3783,3359,66,358,424,...,47.84,3035,548,16.31,827,17,0,0,13,735d9f0b14
1,Mazda - MAZDHOC7W495 - Newsletter Januari - FR...,E-news janvier 2018 : Salon de l’Auto + Action...,Mazda Newsletter Januari FR,"Jan 12, 2018 06:49 pm",Friday,4174,4007,50,117,167,...,53.26,3936,548,13.68,811,11,0,0,13,99f35e0849
2,Mazda - MAZDHOC7W495 - Newsletter Januari - FR...,E-news janvier 2018 : Salon de l’Auto + 3 nouv...,Mazda Newsletter Januari FR,"Jan 12, 2018 06:47 pm",Friday,353,348,1,4,5,...,58.62,373,77,22.13,104,9,0,0,13,3c38f3d56b
3,Mazda - MAZDHOC7W495 - Newsletter Januari - NL...,E-news januari 2018: Saloncondities + Win een ...,Mazda Newsletter Januari NL,"Jan 12, 2018 06:44 pm",Friday,6957,6346,162,449,611,...,48.93,6070,924,14.56,1356,37,0,0,13,e75b474c96
4,Mazda - MAZDHOC7W495 - Newsletter Januari - NL...,E-news januari 2018: Saloncondities + 3 primeurs!,Mazda Newsletter Januari NL,"Jan 12, 2018 06:42 pm",Friday,2635,2487,25,123,148,...,48.77,2393,400,16.08,607,18,1,0,13,8c08c98353


In [13]:
len(set(df_campaigns['List']))

58

In [14]:
# Create a dictionary for the number of campaigns per list
dict_campaign_per_list = {}
for i in list(set(df_campaigns['List'])):
    temp_var = i.lower()
    dict_campaign_per_list[temp_var] = df_campaigns.groupby("List").count()['Title'][i]

## Lists

For each campaign list, there are three files:
- cleaned: Cleaned contacts have addresses that have hard bounced, or repeatedly soft bounced, and are considered invalid.
- members
- unsubscribed

In [15]:
#filecount
print("# of files in unsubscribed :", filecount(os.fspath(master_folder + lists + "unsubscribed/")))
print("# of files in members :", filecount(os.fspath(master_folder + lists + "members/")))
print("# of files in cleaned :", filecount(os.fspath(master_folder + lists + "cleaned/")))

# of files in unsubscribed : 4
# of files in members : 4
# of files in cleaned : 4


In [16]:
files_lists = [f for f in os.listdir(master_folder + lists +  "unsubscribed/")]

In [17]:
df_list_rand = pd.read_csv(master_folder + lists +  "unsubscribed/" + files_lists[0] ,\
                  low_memory=False)

In [18]:
df_list_rand.columns

Index(['Email Address', 'First Name', 'Last Name', 'Type customer',
       'Wedstrijd', 'MEMBER_RATING', 'OPTIN_TIME', 'OPTIN_IP', 'CONFIRM_TIME',
       'CONFIRM_IP', 'LATITUDE', 'LONGITUDE', 'GMTOFF', 'DSTOFF', 'TIMEZONE',
       'CC', 'REGION', 'UNSUB_TIME', 'UNSUB_CAMPAIGN_TITLE',
       'UNSUB_CAMPAIGN_ID', 'UNSUB_REASON', 'UNSUB_REASON_OTHER', 'LEID',
       'EUID', 'NOTES'],
      dtype='object')

In [19]:
df_list_rand.head()

,Email Address,First Name,Last Name,Type customer,Wedstrijd,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,...,CC,REGION,UNSUB_TIME,UNSUB_CAMPAIGN_TITLE,UNSUB_CAMPAIGN_ID,UNSUB_REASON,UNSUB_REASON_OTHER,LEID,EUID,NOTES
0,m.vansteenkiste@me.com,Maarten,Van Steenkiste,Prospect,Zonder wedstrijd,3,2018-01-12 11:48:34,NaN,2018-01-12 11:48:34,NaN,...,NaN,NaN,2018-01-13 08:40:04,Mazda - MAZDHOC7W495 - Newsletter Januari - NL...,cfbab63e82,NaN,NaN,183134549,cbcaf8f21f,NaN
1,kristoflaureys@icloud.com,Kristof,Laureys,Prospect,Zonder wedstrijd,3,2018-01-12 11:48:34,NaN,2018-01-12 11:48:34,NaN,...,NaN,NaN,2018-01-12 19:45:09,Mazda - MAZDHOC7W495 - Newsletter Januari - NL...,cfbab63e82,NaN,NaN,183134557,de87200314,NaN
2,w.j.jans@skynet.be,Wilhelmius,Janssen,Min 3 jaar,Zonder wedstrijd,2,2018-01-12 11:47:46,NaN,2018-01-12 11:47:46,NaN,...,NaN,NaN,2018-01-12 19:40:51,Mazda - MAZDHOC7W495 - Newsletter Januari - NL...,0b3a3f5276,NaN,NaN,183094669,196b4c6fd0,NaN
3,rosiers.karel@gmail.com,Dorothy,Cornette,Min 3 jaar,Zonder wedstrijd,3,2018-01-12 11:47:47,NaN,2018-01-12 11:47:47,NaN,...,NaN,NaN,2018-01-12 19:00:27,Mazda - MAZDHOC7W495 - Newsletter Januari - NL...,0b3a3f5276,NaN,NaN,183095353,24873b871c,NaN
4,vernaillen.wilfried@telenet.be,Wilfried,Vernaillen,Min 3 jaar,Zonder wedstrijd,2,2018-01-12 11:47:48,NaN,2018-01-12 11:47:48,NaN,...,NaN,NaN,2018-01-13 06:49:00,Mazda - MAZDHOC7W495 - Newsletter Januari - NL...,0b3a3f5276,NORMAL,NaN,183096169,ac4caa5b78,NaN


LEID and EUID: LEID is the unique identifier for a contact, specific to a list. EUID is the unique identifier for a contact on the account level, across all lists.

The "UNSUB_CAMPAIGN_ID" refers to the campaign from which he asked to be removed

In [20]:
# Join all the lists together
def join_lists():
    df = {}
    df_join = pd.DataFrame()
    for i in ['unsubscribed/', 'cleaned/', 'members/']:
        df[i] = pd.DataFrame()
        counter = 0
        for j in [f for f in os.listdir(master_folder + lists +  i)]:
            temp_df = pd.read_csv(master_folder + lists +  i + j)
            temp_df['list_name'] = j
            temp_df['type_list'] = i[:-1]
            if counter == 0:
                df[i] = temp_df
                counter += 1
            else:
                df[i] = pd.concat([df[i], temp_df])
                counter += 1
        if counter == 1:
            df_join = df[i]
        else:
            df_join = pd.concat([df_join, df[i]])
    df_join.reset_index(inplace = True)
    
    # Drop columns that always contain na 
    df_join.drop(['index','OPTIN_IP','CONFIRM_IP','NOTES'],\
                      axis=1,inplace = True)
    
    #Add the name of the file in a ne column
    df_join['list_name'] = df_join['list_name']\
                            .apply(lambda x: " ".join(re.split('_|\.',x)[2:-1][0].split('-')))
    
    return df_join

In [21]:
df_all_users = join_lists()

In [22]:
# We will join on the email so we need to make sure the email address column is formatted in the same way
df_all_users['Email Address'] = df_all_users['Email Address'].apply(lambda x: str(x.lower().strip()))
df_all_users['Email Address'] = df_all_users['Email Address'].astype(str)

In [23]:
df_all_users[df_all_users['Email Address'] == 'caro_bellemans@yahoo.com']

,CC,CLEAN_CAMPAIGN_ID,CLEAN_CAMPAIGN_TITLE,CLEAN_TIME,CONFIRM_TIME,DSTOFF,EUID,Email Address,First Name,GMTOFF,...,TIMEZONE,Type customer,UNSUB_CAMPAIGN_ID,UNSUB_CAMPAIGN_TITLE,UNSUB_REASON,UNSUB_REASON_OTHER,UNSUB_TIME,Wedstrijd,list_name,type_list
8520,LU,NaN,NaN,NaN,2018-01-12 11:50:16,2,3f569fc6c8,caro_bellemans@yahoo.com,Caroline,1,...,Europe/Luxembourg,Plus 3 jaar,NaN,NaN,NaN,NaN,NaN,NaN,mazda newsletter januari fr,members
27051,LU,NaN,NaN,NaN,2018-01-08 13:30:56,2,3f569fc6c8,caro_bellemans@yahoo.com,Caroline,1,...,Europe/Luxembourg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mazda cheque lux,members


Fields to be added from lists of users:
- Number of cleaned lists a user belongs to
- Number of unsubscribed lists a user belongs to
- Number of members lists a user belongs to
- EUID
- Type of customer
- Rating of the member

In [24]:
# Check on the duplicates
duplicates = df_cust_data[df_cust_data['Email Address'].isin(df_cust_data['Email Address'].\
                    value_counts()[df_cust_data['Email Address'].value_counts()>1].index)]
len(duplicates[duplicates['Email Address'] != "no email"])/2

16.0

In [25]:
# Aggregate data per user
members = pd.DataFrame(df_all_users[df_all_users['type_list'] == 'members'].\
            groupby('Email Address')['type_list'].count())
cleaned = pd.DataFrame(df_all_users[df_all_users['type_list'] == 'cleaned'].\
            groupby('Email Address')['type_list'].count())
unsubscribed = pd.DataFrame(df_all_users[df_all_users['type_list'] == 'unsubscribed'].\
            groupby('Email Address')['type_list'].count())
EUID = pd.DataFrame(df_all_users.groupby('Email Address')['EUID'].max())

In [26]:
# Create new columns in the mazda dataset for customers
df_cust_data['Chimp_Number_Members_list'] = df_cust_data[['Email Address']].\
                merge(members, how = 'left', left_on = 'Email Address', right_index = True)['type_list']
df_cust_data['Chimp_Number_Clean_list'] = df_cust_data[['Email Address']].\
                merge(cleaned, how = 'left', left_on = 'Email Address', right_index = True)['type_list']
df_cust_data['Chimp_Number_Unsub_list'] = df_cust_data[['Email Address']].\
                merge(unsubscribed, how = 'left', left_on = 'Email Address', right_index = True)['type_list']
df_cust_data['Chimp_EUID'] = df_cust_data[['Email Address']].\
                merge(EUID, how = 'left', left_on = 'Email Address', right_index = True)['EUID']
    
# Create new columns in the mazda dataset for prospects
df_prospects_data['Chimp_Number_Members_list'] = df_prospects_data[['Email Address']].\
                merge(members, how = 'left', left_on = 'Email Address', right_index = True)['type_list']
df_prospects_data['Chimp_Number_Clean_list'] = df_prospects_data[['Email Address']].\
                merge(cleaned, how = 'left', left_on = 'Email Address', right_index = True)['type_list']
df_prospects_data['Chimp_Number_Unsub_list'] = df_prospects_data[['Email Address']].\
                merge(unsubscribed, how = 'left', left_on = 'Email Address', right_index = True)['type_list']
df_prospects_data['Chimp_EUID'] = df_prospects_data[['Email Address']].\
                merge(EUID, how = 'left', left_on = 'Email Address', right_index = True)['EUID']

In [27]:
df_cust_data[df_cust_data['Email Address'] == 'caro_bellemans@yahoo.com']

,Contact Row ID,Preferred Language,Personal Title_I,First Name,Last Name,Street Address,Street Address 2,City,Postal Code,Country_I,...,Current Mileage,Own(ed) a Mazda,Model,MX-5 Lead,MOI,Date,Chimp_Number_Members_list,Chimp_Number_Clean_list,Chimp_Number_Unsub_list,Chimp_EUID
1093,1-3XNUAJ,FRA,Miss,CAROLINE,BELLEMANS,RUE DE CESSANGE 119,NaN,Luxembourg,1321,Luxembourg,...,12552.0,Y,MX-5,NaN,NaN,NaT,2.0,NaN,NaN,3f569fc6c8


In [28]:
sum(df_cust_data['Chimp_EUID'].isnull())

424

In [29]:
len(df_cust_data)

1881

In [30]:
sum(df_prospects_data['Chimp_EUID'].isnull())

5758

In [31]:
len(df_prospects_data)

6738

## Aggregate_activity

The aggregate_activity folder is divided six subsets:
- abuse : list of the users that reported the campaign as an abuse (a user that reported a campaign as a spam)
- bounces : list of users for which the campaign could not be delivered to their email
- click_activity: list of users that have opened the campaign and clicked on the campaign
- not_opened: list of users that have not opened the campaign email
- opened: list of users that have opened the campaign email
- unsub_reasons: list of users that have unsubscribed and a field with the reason

In [32]:
#filecount
print("# of files in abuse :", filecount(os.fspath(master_folder + aggregate_activity + "abuse/")))
print("# of files in bounces :", filecount(os.fspath(master_folder + aggregate_activity + "bounces/")))
print("# of files in click_activity :", filecount(os.fspath(master_folder + aggregate_activity + "click_activity/")))
print("# of files in not_opened :", filecount(os.fspath(master_folder + aggregate_activity + "not_opened/")))
print("# of files in opened :", filecount(os.fspath(master_folder + aggregate_activity + "opened/")))
print("# of files in unsub-reasons :", filecount(os.fspath(master_folder + aggregate_activity + "unsub-reasons/")))

# of files in abuse : 182
# of files in bounces : 182
# of files in click_activity : 182
# of files in not_opened : 182
# of files in opened : 182
# of files in unsub-reasons : 182


In [33]:
files_aggregate = [f for f in os.listdir(master_folder + aggregate_activity +  "abuse/")]

In [34]:
df_agg_abuse_rand = pd.read_csv(master_folder + aggregate_activity + \
                  "abuse/" + files_aggregate[0], low_memory=False)
df_agg_bounces_rand = pd.read_csv(master_folder + aggregate_activity + \
                  "bounces/" + files_aggregate[0], low_memory=False)
df_agg_click_rand = pd.read_csv(master_folder + aggregate_activity + \
                  "click_activity/" + files_aggregate[0], low_memory=False)
df_agg_notop_rand = pd.read_csv(master_folder + aggregate_activity + \
                  "not_opened/" + files_aggregate[0], low_memory=False)
df_agg_op_rand = pd.read_csv(master_folder + aggregate_activity + \
                  "opened/" + files_aggregate[0], low_memory=False)
df_agg_unsub_rand = pd.read_csv(master_folder + aggregate_activity + \
                  "unsub-reasons/" + files_aggregate[0], low_memory=False)

We will join on email address but email addresses are encountered in the following columns:

['Adresse email','E-mail', 'E-mail*', 'E-mailadres', 'Email', 'Email Address']

In [35]:
df_agg_abuse_rand.head()

,Email Address,First Name,Last Name,Member Rating
0,Pdorban@Hotmail.Com,Pascal,Dorban,1


List of users that reported the campaign as an abuse

In [36]:
df_agg_bounces_rand.head()

,Email Address,First Name,Last Name,Member Rating,Bounce Type
0,Burnhampm@Me.Dotcom,Peter,Burnham,1,soft
1,Majstrorski07@Gmail.Com,Mile,Majstorski,1,hard
2,Norbertwebwer1960@Gmail.Com,Norbert,Weber,1,hard
3,Zsuzsanna.Brassai@Europarl.Europa,Zsuzsanna,Brassai,1,soft
4,Christian.Mosbach@Wolter-Wasserbilli.Com,Ewald,Wolter,1,soft


List of users for which the campaign did not reach them

In [37]:
df_agg_click_rand.head()

,Email Address,First Name,Last Name,Member Rating,Clicks
0,Hhildgen@Pt.Lu,Henriette,Hildgen,4,1
1,Leonardocidalia@Hotmail.Fr,Azevedo Santana,Paulo Dinis,4,1
2,Munhoven@Gmail.Com,Tom,Munhoven,4,1
3,Geje@Pt.Lu,Jeannot,Geib,4,1
4,Pfreis@Pt.Lu,Patrick,Freis,4,1


List of users that have given a click for the campaign, with the total number of clicks given

In [38]:
df_agg_notop_rand.head()

,Email Address,First Name,Last Name,Member Rating
0,Benjaminbamberg@Yahoo.De,Benjamin,Bamberg,2
1,Ana8Costa@Gmail.Com,Ana Maria,Alves Da Costa,2
2,Lucja_Rokita@Yahoo.Co.Uk,Lucja,Tryburcy-Rokita,2
3,Broyer.Valery@Neuf.Fr,Sophie,Arvieux,2
4,Georges.Frieden@Gmx.Net,Georges,Frieden,2


List of users that have not open the campaign

In [39]:
df_agg_op_rand.head()

,Email Address,First Name,Last Name,Member Rating,Opens
0,Kirch.Jeannot@Gmail.Com,Jeannot,Kirch,3,2
1,Saimen.Thoma@Tango.Lu,Danielle,Thoma,3,3
2,Malou27@Pt.Lu,Marie-Louise,Eschbour,3,2
3,Hhildgen@Pt.Lu,Henriette,Hildgen,4,11
4,Kiss.Ionut@Yahoo.Com,Ionut,Kiss,3,1


List of users that have opened the campaign and number of times they have opened it

In [40]:
df_agg_unsub_rand.head()

,Email Address,First Name,Last Name,Member Rating,reason,description
0,Ana-Bilic@Hotmail.Com,Ana,Bilic,3,NaN,NaN
1,Rene.Moonen@Education.Lu,Myriam,Moonen--Schwartz,2,NORMAL,NaN
2,Predrag.Latinovic@Gmail.Com,Predrag,Latinovic,3,NOSIGNUP,NaN
3,Pdorban@Hotmail.Com,Pascal,Dorban,1,SPAM,NaN


List of users that have unsubscribed from a campaign

In [41]:
# Join all the lists together
def join_aggregates():
    df = {}
    df_join = pd.DataFrame()
    for i in ['abuse/', 'bounces/', 'click_activity/', 'not_opened/', 'opened/', 'unsub-reasons/']:
        df[i] = pd.DataFrame()
        counter = 0
        for j in [f for f in os.listdir(master_folder + aggregate_activity +  i)]:
            temp_df = pd.read_csv(master_folder + aggregate_activity +  i + j)
            temp_df['campaign_name'] = j
            temp_df['type_aggregate'] = i[:-1]
            if counter == 0:
                df[i] = temp_df
                counter += 1
            else:
                df[i] = pd.concat([df[i], temp_df])
                counter += 1
        if counter == 1:
            df_join = df[i]
        else:
            df_join = pd.concat([df_join, df[i]])
            
    df_join.reset_index(inplace = True)
    
    df_join['Email Address'] = df_join[['Adresse email','E-mail', 'E-mail*', 'E-mailadres', 'Email', 'Email Address']].fillna('')\
        .apply(lambda x: ''.join(x), axis = 1)
    
    df_join['Last Name'] = df_join[['Familienaam','Last Name','Last Name.1','Naam', 'Nom']].fillna('')\
    .apply(lambda x: ''.join(x), axis = 1)
    
    df_join['First Name'] = df_join[['First Name','First Name.1','Voornaam', 'Prénom']].fillna('')\
    .apply(lambda x: ''.join(x), axis = 1)
        
    df_join.drop(['index', 'Aanspreking', 'Adresse email', 'Automodel', 'Automodel URL',
       'Birthdate', 'Brochure', 'Bus', 'Cheque', 'Civilité','Combien de participants',
       'Combien de personnes vont participé à ce concours?', 'Country',
       'Date de naissance', "Date de validité de la carte d'identité",
       'E-mail', 'E-mail*', 'E-mailadres', 'Email',
       'Email Opt-in',
       'GENERATION', 'GENERATION_2','Geldigheidsdatum ID', 'Hoeveel deelnemers gaven het juiste antwoord op he',
       'Hoeveel mensen zullen deelnemen aan de wedstrijd?',
       'Hoeveel mensen zullen deelnemen aan de wedstrijd?*', 'Huisnummer',
       'Identiteitskaartnummer', 'Ik kom:', 'LANGUAGE', 'Land', 'Language',
       'Loser', 'MODEL','Model',
       'Nieuwsbrief','Nuitée près de Charleroi Airport',
       'Nummer', 'Numéro de téléphone', "N° de carte d'identité",
       'Overnachting Charleroi Airport', 'Phone', 'Postcode',
       'Quelle est le terme japonais pour « artisan » ?',
       'Question du concours:', 'Reglement', 'STATUS', 'Segment',
       'Segment_group', 'Stad', 'Status', 'Straat', 'Taal', 'Telefoon',
       'Telefoonnummer', 'Test', 'Title', 'Téléphone', 'Untitled', 'Vegetarisch',
       'Végétarien(ne)', 'Wat is het Japanse woord voor ‘ambachtslui’?',
       'Wat is het Japanse woord voor ‘ambachtslui’?*', 'Wedstrijd',
       'Wedstrijd reglement', 'Wedstrijdvraag:', 'country',
       'description', 'geslacht', 'optin', 'reason', 'tiebreaker',\
       'Familienaam','Last Name.1','Naam', 'Nom', 'First Name.1','Voornaam', 'Prénom',\
        'Type', 'Type customer', 'Type of customer', 'Geboortedatum'],\
                 axis = 1, inplace = True)
    
    return df_join

In [42]:
aggregates = join_aggregates()

In [43]:
aggregates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357708 entries, 0 to 357707
Data columns (total 10 columns):
Bounce Type       21798 non-null object
Clicks            40952 non-null object
Email Address     357708 non-null object
First Name        357708 non-null object
Gender            67699 non-null object
Last Name         357708 non-null object
Member Rating     357708 non-null object
Opens             146154 non-null object
campaign_name     357708 non-null object
type_aggregate    357708 non-null object
dtypes: object(10)
memory usage: 27.3+ MB


In [44]:
# We will join on the email so we need to make sure the email address column is formatted in the same way
aggregates['Email Address'] = aggregates['Email Address'].apply(lambda x: str(x.lower().strip()))
aggregates['Email Address'] = aggregates['Email Address'].astype(str)

In [44]:
len(list(set(aggregates['campaign_name'])))

182

Fields to be added from aggregate activity:

- Total campaigns received 


- Total campaigns opened once
- Total campaigns not opened
- Ratio (Total opened once)/(Total campaigns received)


- Total abuse
- Ratio (Total abuse)/(Total campaigns received) 


- Total clicks
- Ratio (Total clicks)/(Total campaigns opened once)


- Total bounces
- Ratio (Total bounces)/(Total campaigns received)


- Total unsub
- Ratio (Total unsub)/(Total campaigns received)

In [103]:
# Aggregate data per user
Total_email_received = pd.DataFrame(aggregates.groupby(['Email Address', 'campaign_name'])\
                                    ['campaign_name'].nunique())
Total_email_received['count'] = Total_email_received['campaign_name']
Total_email_received.drop('campaign_name', axis = 1, inplace = True)
Total_email_received = pd.DataFrame(Total_email_received.reset_index().groupby('Email Address')['count'].sum())

opened_once = pd.DataFrame(aggregates[(aggregates['type_aggregate'] == 'opened')].\
            groupby('Email Address')['campaign_name'].count())
opened_multi = pd.DataFrame(aggregates[(aggregates['type_aggregate'] == 'opened')].\
            groupby('Email Address')['Opens'].sum()) 
not_opened = pd.DataFrame(aggregates[aggregates['type_aggregate'] == 'not_opened'].\
            groupby('Email Address')['campaign_name'].count())
abuse = pd.DataFrame(aggregates[(aggregates['type_aggregate'] == 'abuse')].\
            groupby('Email Address')['campaign_name'].count())
clicks = pd.DataFrame(aggregates[(aggregates['type_aggregate'] == 'click_activity')].\
            groupby('Email Address')['Clicks'].sum())
bounces = pd.DataFrame(aggregates[(aggregates['type_aggregate'] == 'bounces')].\
            groupby('Email Address')['Bounce Type'].count())
unsub = pd.DataFrame(aggregates[(aggregates['type_aggregate'] == 'unsub-reasons')].\
            groupby('Email Address')['Last Name'].count())
Member_Rating = pd.DataFrame(aggregates.groupby('Email Address')['Member Rating'].last())

In [104]:
# Create new columns in the mazda dataset for customers
df_cust_data['Chimp_Total_Emails'] = df_cust_data[['Email Address']].\
    merge(Total_email_received, how = 'left', left_on = 'Email Address', right_index = True)['count']
df_cust_data['Chimp_opened_once'] = df_cust_data[['Email Address']].\
    merge(opened_once, how = 'left', left_on = 'Email Address', right_index = True)['campaign_name']
df_cust_data['Chimp_opened_multi'] = df_cust_data[['Email Address']].\
    merge(opened_multi, how = 'left', left_on = 'Email Address', right_index = True)['Opens']
df_cust_data['Chimp_not_opened'] = df_cust_data[['Email Address']].\
    merge(not_opened, how = 'left', left_on = 'Email Address', right_index = True)['campaign_name']
df_cust_data['Chimp_abuse'] = df_cust_data[['Email Address']].\
    merge(abuse, how = 'left', left_on = 'Email Address', right_index = True)['campaign_name'] 
df_cust_data['Chimp_clicks'] = df_cust_data[['Email Address']].\
    merge(clicks, how = 'left', left_on = 'Email Address', right_index = True)['Clicks'] 
df_cust_data['Chimp_bounces'] = df_cust_data[['Email Address']].\
    merge(bounces, how = 'left', left_on = 'Email Address', right_index = True)['Bounce Type'] 
df_cust_data['Chimp_unsub'] = df_cust_data[['Email Address']].\
    merge(unsub, how = 'left', left_on = 'Email Address', right_index = True)['Last Name'] 
df_cust_data['Chimp_member_rating'] = df_cust_data[['Email Address']].\
    merge(Member_Rating, how = 'left', left_on = 'Email Address', right_index = True)['Member Rating']

# Create new columns in the mazda dataset for prospects
df_prospects_data['Chimp_Total_Emails'] = df_prospects_data[['Email Address']].\
    merge(Total_email_received, how = 'left', left_on = 'Email Address', right_index = True)['count']
df_prospects_data['Chimp_opened_once'] = df_prospects_data[['Email Address']].\
    merge(opened_once, how = 'left', left_on = 'Email Address', right_index = True)['campaign_name']
df_prospects_data['Chimp_opened_multi'] = df_prospects_data[['Email Address']].\
    merge(opened_multi, how = 'left', left_on = 'Email Address', right_index = True)['Opens']
df_prospects_data['Chimp_not_opened'] = df_prospects_data[['Email Address']].\
    merge(not_opened, how = 'left', left_on = 'Email Address', right_index = True)['campaign_name']
df_prospects_data['Chimp_abuse'] = df_prospects_data[['Email Address']].\
    merge(abuse, how = 'left', left_on = 'Email Address', right_index = True)['campaign_name'] 
df_prospects_data['Chimp_clicks'] = df_prospects_data[['Email Address']].\
    merge(clicks, how = 'left', left_on = 'Email Address', right_index = True)['Clicks'] 
df_prospects_data['Chimp_bounces'] = df_prospects_data[['Email Address']].\
    merge(bounces, how = 'left', left_on = 'Email Address', right_index = True)['Bounce Type'] 
df_prospects_data['Chimp_unsub'] = df_prospects_data[['Email Address']].\
    merge(unsub, how = 'left', left_on = 'Email Address', right_index = True)['Last Name'] 
df_prospects_data['Chimp_member_rating'] = df_prospects_data[['Email Address']].\
    merge(Member_Rating, how = 'left', left_on = 'Email Address', right_index = True)['Member Rating']

In [108]:
df_cust_data[['Email Address','Chimp_Total_Emails','Chimp_opened_once','Chimp_opened_multi','Chimp_not_opened',\
                  'Chimp_abuse', 'Chimp_clicks', 'Chimp_bounces', 'Chimp_unsub', 'Chimp_member_rating']].\
            to_csv('chimp_global_cust')

df_prospects_data[['Email Address','Chimp_Total_Emails','Chimp_opened_once','Chimp_opened_multi','Chimp_not_opened',\
                  'Chimp_abuse', 'Chimp_clicks', 'Chimp_bounces', 'Chimp_unsub', 'Chimp_member_rating']].\
            to_csv('chimp_global_prosp')

## Creation of two cross-tables of emails (rows) per campaigns(columns)

In [109]:
# New Table with all email users as rows and email campaign as columns for customers dataset
df_one_hot_campaign_cust = pd.DataFrame(columns = list(set(aggregates['campaign_name'])),\
                                   index = df_cust_data['Email Address'])

# New Table with all email users as rows and email campaign as columns for prospects dataset
df_one_hot_campaign_prosp = pd.DataFrame(columns = list(set(aggregates['campaign_name'])),\
                                   index = df_prospects_data['Email Address'])

# Fill the above tables with the following formula. For each match of user with a campaign, it will be:
# ---> opens + 2 * Clicks
for i in  list(set(aggregates['campaign_name'])): 
    
    temp_df = pd.DataFrame(aggregates[(aggregates['campaign_name']== i)])\
            .groupby('Email Address')[['Opens','Clicks']].sum()
    temp_df.fillna(0, inplace = True)
    temp_df["temp_sum"] = temp_df['Opens'] + temp_df['Clicks'] * 2
    temp_df = pd.DataFrame(temp_df["temp_sum"])
    
    df_one_hot_campaign_cust = df_one_hot_campaign_cust.merge(temp_df, how = 'left', left_index = True, right_index = True)

    # replace the new column with the one referring to the campaign, replace NA with 0 (user has not received the
    # the campaign) and remove the new column from the merge.
    df_one_hot_campaign_cust[i] = df_one_hot_campaign_cust['temp_sum']
    df_one_hot_campaign_cust[i].fillna(0, inplace = True)
    df_one_hot_campaign_cust = df_one_hot_campaign_cust.iloc[:,:len(list(set(aggregates['campaign_name'])))]

    df_one_hot_campaign_prosp = df_one_hot_campaign_prosp.merge(temp_df, how = 'left', left_index = True, right_index = True)

    # replace the new column with the one referring to the campaign, replace NA with 0 (user has not received the
    # the campaign) and remove the new column from the merge.
    df_one_hot_campaign_prosp[i] = df_one_hot_campaign_prosp['temp_sum']
    df_one_hot_campaign_prosp[i].fillna(0, inplace = True)
    df_one_hot_campaign_prosp = df_one_hot_campaign_prosp.iloc[:,:len(list(set(aggregates['campaign_name'])))]

## Topic Modelling

In [110]:
# Load the list of topics created manually
df = pd.read_excel("mails taggen.xlsx")

In [111]:
# Select only the columns with the tags and put them in the 'tags' dataframe
tags = df.loc[:, 'tags --> ...':].apply(lambda x: x.dropna().tolist(), axis=1)
# Select the other columns and put them in the 'df' dataframe
df = df.drop(df.columns.to_series()['tags --> ...':], axis=1)
# format the tags (remove whitespace and insert underscore instead of white spaces)
tags = tags.apply(lambda x: list(map(lambda y: '_'.join(y.lower().strip().split()), x)))
# insert the tags into the df dataframe
df['tags'] = tags

In [112]:
# Create a set with the list of all unique tags
topics = set()
df['tags'].apply(lambda x: topics.update(x))
topics = tuple(topics)
topics = sorted(topics)
print('topics updated and sorted')

topics updated and sorted


In [113]:
# Create a column for each topic in the df dataframe
for t in topics:
    df[t] = False

In [114]:
# Insert 1 or 0 if the topics appear in the campaign
df.loc[:, topics[0]:topics[-1]] = df.apply(lambda x: \
                                           pd.Series([t in x['tags'] for t in topics], index=topics), axis=1)

In [115]:
# Remove the ".csv" from the file name
df["mail ID"] = df["mail ID"].apply(lambda x: x.split('.')[0])
# Set the index as the 'mail ID'
df = df.set_index("mail ID")

In [116]:
def sumTopics(newsletters):
    topics_opened = [0] * len(topics)
    for newsletter in newsletters:
        index = newsletter.split('.')[0]
        try:
            topics_opened += df.loc[index, topics[0]:topics[-1]].apply(int)
        except:
            print("the newsletter {} does not exist, it has been skipped".format(index))
    return pd.Series(topics_opened, index=topics)

In [117]:
def getNewsletterList(newsletters_is_received):
    return newsletters_is_received[newsletters_is_received==1].index

In [118]:
def getTopics(row):
    newsletters = getNewsletterList(row)
    return sumTopics(newsletters)

In [119]:
def parseAndComputeTopics(df):
    res_df = pd.DataFrame(index=df.index, columns=topics)
    res_df.loc[:, topics[0]:topics[-1]] = df.apply(getTopics, axis=1)
    return res_df

In [120]:
new_cols_cust = parseAndComputeTopics(df_one_hot_campaign_cust)
new_cols_prosp = parseAndComputeTopics(df_one_hot_campaign_prosp)

In [121]:
new_cols_cust.to_csv('tags_cust_proc')
new_cols_prosp.to_csv('tags_prosp_proc')

### Need to exclude emails that were receveied after the purchase date of the car

In [236]:
df_campaigns['Send Date'] = pd.to_datetime(df_campaigns['Send Date'])

In [244]:
df_campaigns['Send Date'][0]

Timestamp('2018-01-12 18:51:00')

In [286]:
df_cust_data['Registration Date'] = pd.to_datetime(df_cust_data['Registration Date'])

In [287]:
df_cust_data['Registration Date'] = df_cust_data['Registration Date'].as_matrix()

In [377]:
#function to find the row in the campaign master file based on name of individual files in other folder

def format_cust_campaign(x):
    file = re.split("[_\-.]",x.lower())
    file = file[1:-1]
    
    return file

In [379]:
test = pd.Series(list(set(aggregates['campaign_name']))).apply(lambda x: format_cust_campaign(x))

In [385]:
arg = []
for i in test:
    argmax = df_campaigns['Title'].apply(lambda x: \
                                         len(list(set(re.split("[ _\-.]",x.lower())).intersection(i)))).argmax()
    arg.append(argmax)

In [393]:
df_campaigns['Title'][32]

'Nieuwsbrief Mei Prospect + Customer >3y LUXFR (copy 02)'